<a href="https://colab.research.google.com/github/SeyfGoumeida/Scorecard_Quotidienne/blob/main/Attente_en_caisse_par_quart_d'heure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Attente en caisse par quart d'heure

## - Charger les fichiers et les renommer de cette façon :
- Charge_Caisse_1.txt
- Charge_Caisse_2.txt
- export{Semaine}.txt
- WEX.csv
- Transco.txt
- Attente.csv



# Scorecard quotidienne Darwin + FU:


In [ ]:
#---------- merci de saisir le numéro de la semaine et le jour -------------

semaine = "5"         
jour = "Dimanche"

#----------------------------------------------------------------

from google.colab import auth
import gspread
import pandas as pd
import matplotlib.pyplot as plt
import csv
from datetime import datetime
pd.set_option("display.max_columns", 100)
#------------------------------------------------------------------------------------------
link1 = "/content/Charge_Caisse_1.txt"
#link1 = "/content/drive/MyDrive/Direction Encaissement/{}/Charge_Caisse_1.txt".format(semaine)

link2 = "/content/Charge_Caisse_2.txt"
#link2 = "/content/drive/MyDrive/Direction Encaissement/{}/Charge_Caisse_2.txt".format(semaine)

#linkcharge3 = "/content/Charge_Caisse_3.txt"
#link2 = "/content/drive/MyDrive/Direction Encaissement/{}/Charge_Caisse_2.txt".format(semaine)

link3 = "/content/export{}.txt".format(semaine)
#link3 = "/content/drive/MyDrive/Direction Encaissement/{}/export{}.txt".format(semaine,semaine)

link4 = "/content/Transco.txt"
#link4 = "/content/drive/MyDrive/Direction Encaissement/Transco.txt"

link5 = "/content/WEX.csv"
#link5 = "/content/drive/MyDrive/Direction Encaissement/{}/{}/WEX.csv".format(semaine,jour)

charge1               = pd.read_csv(link1, sep=";", header = None)
charge2               = pd.read_csv(link2, sep=";", header = None)
#charge3               = pd.read_csv(linkcharge3, sep=";", header = None)

#frames = [charge1, charge2,charge3] 
frames = [charge1, charge2] 
charge = pd.concat(frames)
planif                = pd.read_csv(link3, sep="\t", header = None)
Transco               = pd.read_csv(link4, sep=";", header = None)
WEX                   = pd.read_csv(link5, sep=";", header = None,engine='python',encoding='latin-1')

#------------------------------------------------------------------------------------------
WEX = WEX.astype({23: float,14:float})
for i in range(0,len(WEX[[23]])):
  if((WEX.at[i,23]+WEX.at[i,14])>0):
    WEX.at[i, 'taux sco'] = WEX.at[i,23]/(WEX.at[i,23]+WEX.at[i,14])
  else:
    WEX.at[i, 'taux sco'] =0
#------------------------------------------------------------------------------------------

planif = planif.rename(columns={0:"mag_cod_magasin"})
planif = planif.rename(columns={4:"DEBUT"})

charge = charge.rename(columns={0:"anabel"})
charge = charge.rename(columns={3:"Heure de début"})

#-----------------------------------------------------------------------------------------
for i in range(0,len(planif[['mag_cod_magasin']])):
  if len(planif.at[i, "DEBUT"])== 5:
    planif.at[i, "DEBUT"] = planif.at[i, "DEBUT"]+":00"
  if len(planif.at[i, 5])== 5:
    planif.at[i, 5] = planif.at[i, 5]+":00"
#------------------------------------------------------------------------------------------

nb_caisses_SCO1=[]
for i in WEX[23]:
  if i==0:
    nb_caisses_SCO1.append(0)
  else:
    if i <= 6:
      nb_caisses_SCO1.append(1)
    else:
      nb_caisses_SCO1.append(2)

#---------------------------------------------------

Caisse_Ouverte = WEX[[1,3,4,5,14,23,"taux sco"]]
#---------------------------------------------------

Caisse_Ouverte["nb_caisses_SCO1"]= nb_caisses_SCO1
#---------------------------------------------------
Nb_caisses_ouvertes=[]
for i in range(0,len(WEX)):
  Nb_caisses_ouvertes.append(Caisse_Ouverte["nb_caisses_SCO1"][i]+Caisse_Ouverte[14][i])
#---------------------------------------------------

Caisse_Ouverte["Nb_caisses_ouvertes"]= Nb_caisses_ouvertes


Caisse_Ouverte = Caisse_Ouverte.rename(columns={1:"stoAnabelKey",3:"dataKey",4:"jour_semaine",5:"hour_trx",14:"nb_caisses_tradi",23:"nb_caisses_SCO"})

#Caisse_Ouverte.to_csv(r'Caisses_ouvertes.csv', index = False, header=True)

#------------------------------------------------------------------------------------------
tab1= pd.merge(charge,planif,how='inner',left_on=['anabel','Heure de début'],right_on=['mag_cod_magasin','DEBUT'])
tab2= pd.merge(Transco,tab1,how='inner',left_on=[1],right_on=['1_y'])
tab3 = pd.merge(Caisse_Ouverte,tab2,how='inner',left_on=["jour_semaine","hour_trx","dataKey","stoAnabelKey"],right_on=[0,"Heure de début","2_x","anabel"])
tab3.sort_values(by=['mag_cod_magasin', 'jour_semaine','DEBUT'])
df = tab3[['mag_cod_magasin','jour_semaine','DEBUT',4,6,6,6,6,6,'Nb_caisses_ouvertes',6,"taux sco"]]
df = df.set_axis(['Anabel', 'Jour','Heure','Caisses_Prevues','Caisses_Planifiees','Ecart_en_Nb_de_Caisses','Prevues_en_H','Planifiees_en_H','Ecart_en_H','Nb_de_Caisses_Ouvertes','Caisses_Ouvertes_en_H','taux sco'], axis=1)
#--------------------------------------------------------------------------------------------
for i in range(0,len(df[['Anabel']])):
  if isinstance(df.at[i, 'Caisses_Prevues'], str):
    df.at[i, 'Caisses_Prevues'] = df.at[i, 'Caisses_Prevues'].replace(",",".")
sous_couverture = []
sur_couverture = []
sur_sous_couverture = []
taux_sous_couverture = []
taux_sur_couverture = []

for i in range(0,len(df[['Anabel']])):

  df = df.astype({"Caisses_Prevues": float,"Caisses_Planifiees":float,"Ecart_en_Nb_de_Caisses": float,"Prevues_en_H":float,"Planifiees_en_H": float,"Ecart_en_H":float,"Nb_de_Caisses_Ouvertes": float,"Caisses_Ouvertes_en_H":float})
  #-----------------------------------------------------------------------------------------------
  df.at[i, 'Ecart_en_Nb_de_Caisses'] = df.at[i, 'Caisses_Planifiees']- df.at[i, 'Caisses_Prevues']
  df.at[i, 'Prevues_en_H'] = df.at[i, 'Caisses_Prevues']*0.25
  df.at[i, 'Planifiees_en_H'] = df.at[i, 'Caisses_Planifiees']*0.25
  df.at[i, 'Ecart_en_H'] = df.at[i, 'Planifiees_en_H']- df.at[i, 'Prevues_en_H']
  df.at[i, 'Caisses_Ouvertes_en_H'] = df.at[i, 'Nb_de_Caisses_Ouvertes']*0.25
  #-----------------------------------------------------------------------------------------------
  if (df.at[i, 'Prevues_en_H'] - df.at[i, 'Caisses_Ouvertes_en_H'] > 0):
    sur_couverture.append(0)
    taux_sur_couverture.append(0)
    sous_couverture.append(df.at[i, 'Prevues_en_H'] - df.at[i, 'Caisses_Ouvertes_en_H'])
    sur_sous_couverture.append(-(df.at[i, 'Prevues_en_H'] - df.at[i, 'Caisses_Ouvertes_en_H']))
    taux_sous_couverture.append(sous_couverture[i] / df.at[i, 'Prevues_en_H'] )
  else:
    if  (df.at[i, 'Prevues_en_H'] - df.at[i, 'Caisses_Ouvertes_en_H'] < 0):
      sur_couverture.append(df.at[i, 'Caisses_Ouvertes_en_H'] - df.at[i, 'Prevues_en_H'])
      sur_sous_couverture.append( df.at[i, 'Caisses_Ouvertes_en_H'] - df.at[i, 'Prevues_en_H'])
      taux_sur_couverture.append(sur_couverture[i] / df.at[i, 'Caisses_Ouvertes_en_H'] )
      sous_couverture.append(0)
      taux_sous_couverture.append(0)

    else:
      sur_sous_couverture.append(0)
      sous_couverture.append(0)
      sur_couverture.append(0)
      taux_sur_couverture.append(0)
      taux_sous_couverture.append(0)


df['Ecart_sur_sous_couverture']= sur_sous_couverture
df['Ecart_sous_couverture']= sous_couverture
df['Ecart_sur_couverture']= sur_couverture
df['Taux_sur_couverture']= taux_sur_couverture
df['Taux_sous_couverture']= taux_sous_couverture

#-------------------------------------------------------------------------------------------------
link7 = "/content/Attente.csv"
#link7 = "/content/drive/MyDrive/Direction Encaissement/{}/{}/Attente.csv".format(semaine,jour)

#Attente_en_caisse        = pd.read_csv(link6, sep=",",encoding='latin-1')
Attente_en_caisse_eclate = pd.read_csv(link7, sep=",",encoding="utf-8")

#-----------------------------------------------------------------------------------------
for i in range(0,len(Attente_en_caisse_eclate[['QuarterId']])):
  if len(Attente_en_caisse_eclate.at[i, "QuarterId"])== 5:
    Attente_en_caisse_eclate.at[i, "QuarterId"] = Attente_en_caisse_eclate.at[i, "QuarterId"]+":00"
BDD = pd.merge(df,Attente_en_caisse_eclate,how='left',left_on=['Anabel','Heure'],right_on=['Magasin (code Anabel)','QuarterId'])
BDD = BDD[['Anabel','Magasin','Jour','creationDate','Heure','Caisses_Prevues','Caisses_Planifiees','Ecart_en_Nb_de_Caisses','Prevues_en_H','Planifiees_en_H','Ecart_en_H','Nb_de_Caisses_Ouvertes','Caisses_Ouvertes_en_H','Ecart_sous_couverture','Taux_sous_couverture','Ecart_sur_couverture','Taux_sur_couverture','Ecart_sur_sous_couverture','pointOfSaleSurveyResponseValueInt','taux sco']]
BDD = BDD.rename(columns={"pointOfSaleSurveyResponseValueInt":"attente_en_caisse"})
BDD = BDD.rename(columns={"taux sco":"Taux SCO"})

BDD["3plus"] = BDD["attente_en_caisse"]

for i in range(0,len(BDD[['3plus']])):
  BDD.at[i, "creationDate"]=Attente_en_caisse_eclate.at[0,"creationDate"]
  if BDD.at[i, "3plus"]<3 :
    BDD.at[i, "3plus"] = 0
BDD['attente_en_caisse'] = BDD['attente_en_caisse'].fillna(0)
BDD['3plus'] = BDD['3plus'].fillna(0)


# #date format
# from datetime import datetime
# for k in range(0,len(BDD[['Magasin']])):
#   date = datetime.strptime(str(BDD.at[k, "creationDate"]),'%d %b %Y')	
#   date = datetime.strptime(str(date),"%Y-%m-%d %H:%M:%S").strftime('%d %b. %Y')
#   BDD.at[k, "creationDate"] = date
#-------------------------------------------------------------------------------------------------
linkbdd = 'BDD_{}_{}.csv'.format(semaine,jour)
BDD.to_csv(linkbdd, index = False, header=True,sep=';',encoding="latin-1")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
Attente_en_caisse_eclate

,creationDate,Magasin (code Anabel),Magasin,QuarterId,pointOfSaleSurveyResponseValueInt
0,24 Jan 2022,3277,Market Bonneval Louveterie,09:00:00,2
1,24 Jan 2022,3277,Market Bonneval Louveterie,09:15:00,3
2,24 Jan 2022,3277,Market Bonneval Louveterie,09:30:00,1
3,24 Jan 2022,3277,Market Bonneval Louveterie,09:45:00,0
4,24 Jan 2022,3277,Market Bonneval Louveterie,10:00:00,1
...,...,...,...,...,...
11951,24 Jan 2022,9002,Market Beaurains,18:30:00,0
11952,24 Jan 2022,9002,Market Beaurains,18:45:00,1
11953,24 Jan 2022,9002,Market Beaurains,19:00:00,0
11954,24 Jan 2022,9002,Market Beaurains,19:15:00,0


# Scorecard quotidienne Darwin :


In [ ]:
#---------- merci de saisir le numéro de la semaine et le jour -------------

semaine = "3"         
jour = "Dimanche"

#----------------------------------------------------------------

from google.colab import auth
import gspread
import pandas as pd
import matplotlib.pyplot as plt
import csv
from datetime import datetime
pd.set_option("display.max_columns", 100)
#------------------------------------------------------------------------------------------
link1 = "/content/Charge_Caisse_1.txt"
#link1 = "/content/drive/MyDrive/Direction Encaissement/{}/Charge_Caisse_1.txt".format(semaine)

link2 = "/content/Charge_Caisse_2.txt"
#link2 = "/content/drive/MyDrive/Direction Encaissement/{}/Charge_Caisse_2.txt".format(semaine)

#linkcharge3 = "/content/Charge_Caisse_3.txt"
#link2 = "/content/drive/MyDrive/Direction Encaissement/{}/Charge_Caisse_2.txt".format(semaine)

link3 = "/content/export{}.txt".format(semaine)
#link3 = "/content/drive/MyDrive/Direction Encaissement/{}/export{}.txt".format(semaine,semaine)

link4 = "/content/Transco.txt"
#link4 = "/content/drive/MyDrive/Direction Encaissement/Transco.txt"

link5 = "/content/WEX.csv"
#link5 = "/content/drive/MyDrive/Direction Encaissement/{}/{}/WEX.csv".format(semaine,jour)

charge1               = pd.read_csv(link1, sep=";", header = None)
charge2               = pd.read_csv(link2, sep=";", header = None)
#charge3               = pd.read_csv(linkcharge3, sep=";", header = None)

#frames = [charge1, charge2,charge3] 
frames = [charge1, charge2] 
charge = pd.concat(frames)
planif                = pd.read_csv(link3, sep="\t", header = None)
Transco               = pd.read_csv(link4, sep=";", header = None)
WEX                   = pd.read_csv(link5, sep=";", header = None,engine='python')

#------------------------------------------------------------------------------------------
WEX = WEX.astype({23: float,14:float})
for i in range(0,len(WEX[[23]])):
  if((WEX.at[i,23]+WEX.at[i,14])>0):
    WEX.at[i, 'taux sco'] = WEX.at[i,23]/(WEX.at[i,23]+WEX.at[i,14])
  else:
    WEX.at[i, 'taux sco'] =0
#------------------------------------------------------------------------------------------

planif = planif.rename(columns={0:"mag_cod_magasin"})
planif = planif.rename(columns={4:"DEBUT"})

charge = charge.rename(columns={0:"anabel"})
charge = charge.rename(columns={3:"Heure de début"})

#-----------------------------------------------------------------------------------------
for i in range(0,len(planif[['mag_cod_magasin']])):
  if len(planif.at[i, "DEBUT"])== 5:
    planif.at[i, "DEBUT"] = planif.at[i, "DEBUT"]+":00"
  if len(planif.at[i, 5])== 5:
    planif.at[i, 5] = planif.at[i, 5]+":00"
#------------------------------------------------------------------------------------------

nb_caisses_SCO1=[]
for i in WEX[23]:
  if i==0:
    nb_caisses_SCO1.append(0)
  else:
    if i <= 6:
      nb_caisses_SCO1.append(1)
    else:
      nb_caisses_SCO1.append(2)

#---------------------------------------------------

Caisse_Ouverte = WEX[[1,3,4,5,14,23,"taux sco"]]
#---------------------------------------------------

Caisse_Ouverte["nb_caisses_SCO1"]= nb_caisses_SCO1
#---------------------------------------------------
Nb_caisses_ouvertes=[]
for i in range(0,len(WEX)):
  Nb_caisses_ouvertes.append(Caisse_Ouverte["nb_caisses_SCO1"][i]+Caisse_Ouverte[14][i])
#---------------------------------------------------

Caisse_Ouverte["Nb_caisses_ouvertes"]= Nb_caisses_ouvertes


Caisse_Ouverte = Caisse_Ouverte.rename(columns={1:"stoAnabelKey",3:"dataKey",4:"jour_semaine",5:"hour_trx",14:"nb_caisses_tradi",23:"nb_caisses_SCO"})

#Caisse_Ouverte.to_csv(r'Caisses_ouvertes.csv', index = False, header=True)

#------------------------------------------------------------------------------------------
tab1= pd.merge(charge,planif,how='inner',left_on=['anabel','Heure de début'],right_on=['mag_cod_magasin','DEBUT'])
tab2= pd.merge(Transco,tab1,how='inner',left_on=[1],right_on=['1_y'])
tab3 = pd.merge(Caisse_Ouverte,tab2,how='inner',left_on=["jour_semaine","hour_trx","dataKey","stoAnabelKey"],right_on=[0,"Heure de début","2_x","anabel"])
tab3.sort_values(by=['mag_cod_magasin', 'jour_semaine','DEBUT'])
df = tab3[['mag_cod_magasin','jour_semaine','DEBUT',4,6,6,6,6,6,'Nb_caisses_ouvertes',6,"taux sco"]]
df = df.set_axis(['Anabel', 'Jour','Heure','Caisses_Prevues','Caisses_Planifiees','Ecart_en_Nb_de_Caisses','Prevues_en_H','Planifiees_en_H','Ecart_en_H','Nb_de_Caisses_Ouvertes','Caisses_Ouvertes_en_H','taux sco'], axis=1)
#--------------------------------------------------------------------------------------------
for i in range(0,len(df[['Anabel']])):
  if isinstance(df.at[i, 'Caisses_Prevues'], str):
    df.at[i, 'Caisses_Prevues'] = df.at[i, 'Caisses_Prevues'].replace(",",".")
sous_couverture = []
sur_couverture = []
sur_sous_couverture = []
taux_sous_couverture = []
taux_sur_couverture = []

for i in range(0,len(df[['Anabel']])):

  df = df.astype({"Caisses_Prevues": float,"Caisses_Planifiees":float,"Ecart_en_Nb_de_Caisses": float,"Prevues_en_H":float,"Planifiees_en_H": float,"Ecart_en_H":float,"Nb_de_Caisses_Ouvertes": float,"Caisses_Ouvertes_en_H":float})
  #-----------------------------------------------------------------------------------------------
  df.at[i, 'Ecart_en_Nb_de_Caisses'] = df.at[i, 'Caisses_Planifiees']- df.at[i, 'Caisses_Prevues']
  df.at[i, 'Prevues_en_H'] = df.at[i, 'Caisses_Prevues']*0.25
  df.at[i, 'Planifiees_en_H'] = df.at[i, 'Caisses_Planifiees']*0.25
  df.at[i, 'Ecart_en_H'] = df.at[i, 'Planifiees_en_H']- df.at[i, 'Prevues_en_H']
  df.at[i, 'Caisses_Ouvertes_en_H'] = df.at[i, 'Nb_de_Caisses_Ouvertes']*0.25
  #-----------------------------------------------------------------------------------------------
  if (df.at[i, 'Prevues_en_H'] - df.at[i, 'Caisses_Ouvertes_en_H'] > 0):
    sur_couverture.append(0)
    taux_sur_couverture.append(0)
    sous_couverture.append(df.at[i, 'Prevues_en_H'] - df.at[i, 'Caisses_Ouvertes_en_H'])
    sur_sous_couverture.append(-(df.at[i, 'Prevues_en_H'] - df.at[i, 'Caisses_Ouvertes_en_H']))
    taux_sous_couverture.append(sous_couverture[i] / df.at[i, 'Prevues_en_H'] )
  else:
    if  (df.at[i, 'Prevues_en_H'] - df.at[i, 'Caisses_Ouvertes_en_H'] < 0):
      sur_couverture.append(df.at[i, 'Caisses_Ouvertes_en_H'] - df.at[i, 'Prevues_en_H'])
      sur_sous_couverture.append( df.at[i, 'Caisses_Ouvertes_en_H'] - df.at[i, 'Prevues_en_H'])
      taux_sur_couverture.append(sur_couverture[i] / df.at[i, 'Caisses_Ouvertes_en_H'] )
      sous_couverture.append(0)
      taux_sous_couverture.append(0)

    else:
      sur_sous_couverture.append(0)
      sous_couverture.append(0)
      sur_couverture.append(0)
      taux_sur_couverture.append(0)
      taux_sous_couverture.append(0)


df['Ecart_sur_sous_couverture']= sur_sous_couverture
df['Ecart_sous_couverture']= sous_couverture
df['Ecart_sur_couverture']= sur_couverture
df['Taux_sur_couverture']= taux_sur_couverture
df['Taux_sous_couverture']= taux_sous_couverture

#-------------------------------------------------------------------------------------------------
link7 = "/content/Attente.csv"
#link7 = "/content/drive/MyDrive/Direction Encaissement/{}/{}/Attente.csv".format(semaine,jour)

#Attente_en_caisse        = pd.read_csv(link6, sep=",",encoding='latin-1')
Attente_en_caisse_eclate = pd.read_csv(link7, sep=",",encoding="utf-8")

#-----------------------------------------------------------------------------------------
for i in range(0,len(Attente_en_caisse_eclate[['QuarterId']])):
  if len(Attente_en_caisse_eclate.at[i, "QuarterId"])== 5:
    Attente_en_caisse_eclate.at[i, "QuarterId"] = Attente_en_caisse_eclate.at[i, "QuarterId"]+":00"
BDD = pd.merge(df,Attente_en_caisse_eclate,how='inner',left_on=['Anabel','Heure'],right_on=['Magasin (code Anabel)','QuarterId'])
BDD = BDD[['Anabel','Magasin','Jour','creationDate','Heure','Caisses_Prevues','Caisses_Planifiees','Ecart_en_Nb_de_Caisses','Prevues_en_H','Planifiees_en_H','Ecart_en_H','Nb_de_Caisses_Ouvertes','Caisses_Ouvertes_en_H','Ecart_sous_couverture','Taux_sous_couverture','Ecart_sur_couverture','Taux_sur_couverture','Ecart_sur_sous_couverture','pointOfSaleSurveyResponseValueInt','taux sco']]
BDD = BDD.rename(columns={"pointOfSaleSurveyResponseValueInt":"attente_en_caisse"})
BDD = BDD.rename(columns={"taux sco":"Taux SCO"})

BDD["3plus"] = BDD["attente_en_caisse"]

for i in range(0,len(BDD[['3plus']])):
  if BDD.at[i, "3plus"]<3 :
    BDD.at[i, "3plus"] = 0

# #date format
# from datetime import datetime
# for k in range(0,len(BDD[['Magasin']])):
#   date = datetime.strptime(str(BDD.at[k, "creationDate"]),'%d %b %Y')	
#   date = datetime.strptime(str(date),"%Y-%m-%d %H:%M:%S").strftime('%d %b. %Y')
#   BDD.at[k, "creationDate"] = date
#-------------------------------------------------------------------------------------------------
linkbdd = 'BDD_{}_{}.csv'.format(semaine,jour)
BDD.to_csv(linkbdd, index = False, header=True,sep=';',encoding="latin-1")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Sous Couverture réelle et prévu : 

In [ ]:
#---------- merci de saisir le numéro de la semaine -------------

semaine = "08"         
#----------------------------------------------------------------

from google.colab import auth
import gspread
import pandas as pd
import matplotlib.pyplot as plt
import csv
from datetime import datetime
pd.set_option("display.max_columns", 100)
#------------------------------------------------------------------------------------------
link1 = "/content/Charge_Caisse_1.txt"
#link1 = "/content/drive/MyDrive/Direction Encaissement/{}/Charge_Caisse_1.txt".format(semaine)

link2 = "/content/Charge_Caisse_2.txt"
#link2 = "/content/drive/MyDrive/Direction Encaissement/{}/Charge_Caisse_2.txt".format(semaine)

link3 = "/content/export{}.txt".format(semaine)
#link3 = "/content/drive/MyDrive/Direction Encaissement/{}/export{}.txt".format(semaine,semaine)

link4 = "/content/Transco.txt"
#link4 = "/content/drive/MyDrive/Direction Encaissement/Transco.txt"

charge1               = pd.read_csv(link1, sep=";", header = None)
charge2               = pd.read_csv(link2, sep=";", header = None)
frames = [charge1, charge2] 
charge = pd.concat(frames)
planif                = pd.read_csv(link3, sep="\t", header = None)
Transco               = pd.read_csv(link4, sep=";", header = None)

#------------------------------------------------------------------------------------------

planif = planif.rename(columns={0:"mag_cod_magasin"})
planif = planif.rename(columns={4:"DEBUT"})

charge = charge.rename(columns={0:"anabel"})
charge = charge.rename(columns={2:"Date"})
charge = charge.rename(columns={3:"Heure de début"})

#-----------------------------------------------------------------------------------------
for i in range(0,len(planif[['mag_cod_magasin']])):
  if len(planif.at[i, "DEBUT"])== 5:
    planif.at[i, "DEBUT"] = planif.at[i, "DEBUT"]+":00"
  if len(planif.at[i, 5])== 5:
    planif.at[i, 5] = planif.at[i, 5]+":00"
#------------------------------------------------------------------------------------------

tab1= pd.merge(charge,planif,how='inner',left_on=['anabel','Heure de début'],right_on=['mag_cod_magasin','DEBUT'])
tab2= pd.merge(Transco,tab1,how='inner',left_on=[1],right_on=['1_y'])
tab2
#Caisse_Ouverte = Caisse_Ouverte.rename(columns={1:"stoAnabelKey",3:"dataKey",4:"jour_semaine",5:"hour_trx",14:"nb_caisses_tradi",23:"nb_caisses_SCO"})
#tab3 = pd.merge(Caisse_Ouverte,tab2,how='inner',left_on=["jour_semaine","hour_trx","dataKey","stoAnabelKey"],right_on=[0,"Heure de début","2_x","anabel"])
tab2.sort_values(by=['mag_cod_magasin', 0,'DEBUT'])
df = tab2[['mag_cod_magasin',0,0,'Date','DEBUT',4,6,6,6,6,6]]
df = df.set_axis(['Anabel', 'Jour','Jour_correct','Date','Heure','Caisses_Prevues','Caisses_Planifiees','Ecart_en_Nb_de_Caisses','Prevues_en_H','Planifiees_en_H','Ecart_en_H'], axis=1)
#----------------------------------------------------------------------
from datetime import datetime

k = datetime.strptime("10/01/2022",'%d/%m/%Y')	
k = datetime.strptime(str(k),"%Y-%m-%d %H:%M:%S").strftime('%A')

for k in range(0,len(df[['Date']])):
  date = datetime.strptime(str(df.at[k, "Date"]),'%d/%m/%Y')	
  date = datetime.strptime(str(date),"%Y-%m-%d %H:%M:%S").strftime('%A')
  if date == "Monday" : 
     df.at[k, "Jour_correct"] = "Lundi"
  elif date == "Tuesday" :
      df.at[k, "Jour_correct"] = "Mardi"
  elif date == "Wednesday" :
      df.at[k, "Jour_correct"] = "Mercredi"
  elif date == "Thursday" :
      df.at[k, "Jour_correct"] = "Jeudi"
  elif date == "Friday" :
      df.at[k, "Jour_correct"] = "Vendredi"
  elif date == "Saturday" :
      df.at[k, "Jour_correct"] = "Samedi"
  elif date == "Sunday" :
      df.at[k, "Jour_correct"] = "Dimanche"

df = df.drop(df[df.Jour != df.Jour_correct].index)
df = df.drop(['Jour_correct'], axis=1)
#---------------------------------------------------------------------
df = df.reset_index(drop=True)

#---------------------------------------------------------------------
#--------------------------------------------------------------------------------------------
for i in range(0,len(df[['Anabel']])):
  if isinstance(df.at[i, 'Caisses_Prevues'], str):
    df.at[i, 'Caisses_Prevues'] = df.at[i, 'Caisses_Prevues'].replace(",",".")
sous_couverture = []
sur_couverture = []
sur_sous_couverture = []
taux_sous_couverture = []
taux_sur_couverture = []
df = df.astype({"Caisses_Prevues": float,"Caisses_Planifiees":float,"Ecart_en_Nb_de_Caisses": float,"Prevues_en_H":float,"Planifiees_en_H": float,"Ecart_en_H":float})

for i in range(0,len(df[['Anabel']])):

  #-----------------------------------------------------------------------------------------------
  df.at[i, 'Ecart_en_Nb_de_Caisses'] = df.at[i, 'Caisses_Planifiees']- df.at[i, 'Caisses_Prevues']
  df.at[i, 'Prevues_en_H'] = df.at[i, 'Caisses_Prevues']*0.25
  df.at[i, 'Planifiees_en_H'] = df.at[i, 'Caisses_Planifiees']*0.25
  df.at[i, 'Ecart_en_H'] = df.at[i, 'Planifiees_en_H']- df.at[i, 'Prevues_en_H']
  #-----------------------------------------------------------------------------------------------


  if (df.at[i, 'Prevues_en_H'] - df.at[i, 'Planifiees_en_H'] > 0):
    sur_couverture.append(0)
    taux_sur_couverture.append(0)
    sous_couverture.append(df.at[i, 'Prevues_en_H'] - df.at[i, 'Planifiees_en_H'])
    sur_sous_couverture.append(-(df.at[i, 'Prevues_en_H'] - df.at[i, 'Planifiees_en_H']))
    taux_sous_couverture.append(sous_couverture[i] / df.at[i, 'Prevues_en_H'] )
  else:
    if  (df.at[i, 'Prevues_en_H'] - df.at[i, 'Planifiees_en_H'] < 0):
      sur_couverture.append(df.at[i, 'Planifiees_en_H'] - df.at[i, 'Prevues_en_H'])
      sur_sous_couverture.append( df.at[i, 'Planifiees_en_H'] - df.at[i, 'Prevues_en_H'])
      taux_sur_couverture.append(sur_couverture[i] / df.at[i, 'Planifiees_en_H'] )
      sous_couverture.append(0)
      taux_sous_couverture.append(0)

    else:
      sur_sous_couverture.append(0)
      sous_couverture.append(0)
      sur_couverture.append(0)
      taux_sur_couverture.append(0)
      taux_sous_couverture.append(0)


df['Ecart_sur_sous_couverture']= sur_sous_couverture
df['Ecart_sous_couverture']= sous_couverture
df['Ecart_sur_couverture']= sur_couverture
df['Taux_sur_couverture']= taux_sur_couverture
df['Taux_sous_couverture']= taux_sous_couverture

#-------------------------------------------------------------------------------------------------
linkbdd = 'SousCouvPrevu_S{}.csv'.format(semaine)
df = df[['Anabel','Jour','Date','Heure','Caisses_Prevues','Caisses_Planifiees']]
df.to_csv(linkbdd, index = False, header=True,sep=';',encoding="latin-1")

df


,Anabel,Jour,Date,Heure,Caisses_Prevues,Caisses_Planifiees
0,8180,Lundi,21/02/2022,08:30:00,1.0,2.0
1,8980,Lundi,21/02/2022,08:30:00,2.0,2.0
2,9002,Lundi,21/02/2022,08:30:00,1.0,2.0
3,7531,Lundi,21/02/2022,08:30:00,1.0,3.0
4,7647,Lundi,21/02/2022,08:30:00,1.0,2.0
...,...,...,...,...,...,...
77458,7862,Dimanche,27/02/2022,12:45:00,2.0,7.0
77459,7583,Dimanche,27/02/2022,12:45:00,4.0,5.0
77460,7567,Dimanche,27/02/2022,13:00:00,1.0,10.0
77461,7575,Dimanche,27/02/2022,13:00:00,1.0,4.0


# Scorecard quotidienne :

In [ ]:
#---------- merci de saisir le numéro de la semaine et le jour -------------

semaine = "3"         
jour = "Jeudi"

#----------------------------------------------------------------

from google.colab import auth
import gspread
import pandas as pd
import matplotlib.pyplot as plt
import csv
from datetime import datetime
pd.set_option("display.max_columns", 100)
#------------------------------------------------------------------------------------------
link1 = "/content/Charge_Caisse_1.txt"
#link1 = "/content/drive/MyDrive/Direction Encaissement/{}/Charge_Caisse_1.txt".format(semaine)

link2 = "/content/Charge_Caisse_2.txt"
#link2 = "/content/drive/MyDrive/Direction Encaissement/{}/Charge_Caisse_2.txt".format(semaine)

#linkcharge3 = "/content/Charge_Caisse_3.txt"
#link2 = "/content/drive/MyDrive/Direction Encaissement/{}/Charge_Caisse_2.txt".format(semaine)

link3 = "/content/export{}.txt".format(semaine)
#link3 = "/content/drive/MyDrive/Direction Encaissement/{}/export{}.txt".format(semaine,semaine)

link4 = "/content/Transco.txt"
#link4 = "/content/drive/MyDrive/Direction Encaissement/Transco.txt"

link5 = "/content/WEX.csv"
#link5 = "/content/drive/MyDrive/Direction Encaissement/{}/{}/WEX.csv".format(semaine,jour)

charge1               = pd.read_csv(link1, sep=";", header = None)
charge2               = pd.read_csv(link2, sep=";", header = None)
#charge3               = pd.read_csv(linkcharge3, sep=";", header = None)

#frames = [charge1, charge2,charge3] 
frames = [charge1, charge2] 
charge = pd.concat(frames)
planif                = pd.read_csv(link3, sep="\t", header = None)
Transco               = pd.read_csv(link4, sep=";", header = None)
WEX                   = pd.read_csv(link5, sep=";", header = None,engine='python')

#------------------------------------------------------------------------------------------
WEX = WEX.astype({23: float,14:float})
for i in range(0,len(WEX[[23]])):
  if((WEX.at[i,23]+WEX.at[i,14])>0):
    WEX.at[i, 'taux sco'] = WEX.at[i,23]/(WEX.at[i,23]+WEX.at[i,14])
  else:
    WEX.at[i, 'taux sco'] =0
#------------------------------------------------------------------------------------------

planif = planif.rename(columns={0:"mag_cod_magasin"})
planif = planif.rename(columns={4:"DEBUT"})

charge = charge.rename(columns={0:"anabel"})
charge = charge.rename(columns={3:"Heure de début"})

#-----------------------------------------------------------------------------------------
for i in range(0,len(planif[['mag_cod_magasin']])):
  if len(planif.at[i, "DEBUT"])== 5:
    planif.at[i, "DEBUT"] = planif.at[i, "DEBUT"]+":00"
  if len(planif.at[i, 5])== 5:
    planif.at[i, 5] = planif.at[i, 5]+":00"
#------------------------------------------------------------------------------------------

nb_caisses_SCO1=[]
for i in WEX[23]:
  if i==0:
    nb_caisses_SCO1.append(0)
  else:
    if i <= 6:
      nb_caisses_SCO1.append(1)
    else:
      nb_caisses_SCO1.append(2)

#---------------------------------------------------

Caisse_Ouverte = WEX[[1,3,4,5,14,23,"taux sco"]]
#---------------------------------------------------

Caisse_Ouverte["nb_caisses_SCO1"]= nb_caisses_SCO1
#---------------------------------------------------
Nb_caisses_ouvertes=[]
for i in range(0,len(WEX)):
  Nb_caisses_ouvertes.append(Caisse_Ouverte["nb_caisses_SCO1"][i]+Caisse_Ouverte[14][i])
#---------------------------------------------------

Caisse_Ouverte["Nb_caisses_ouvertes"]= Nb_caisses_ouvertes


Caisse_Ouverte = Caisse_Ouverte.rename(columns={1:"stoAnabelKey",3:"dataKey",4:"jour_semaine",5:"hour_trx",14:"nb_caisses_tradi",23:"nb_caisses_SCO"})

#Caisse_Ouverte.to_csv(r'Caisses_ouvertes.csv', index = False, header=True)

#------------------------------------------------------------------------------------------
tab1= pd.merge(charge,planif,how='inner',left_on=['anabel','Heure de début'],right_on=['mag_cod_magasin','DEBUT'])
tab2= pd.merge(Transco,tab1,how='inner',left_on=[1],right_on=['1_y'])
tab3 = pd.merge(Caisse_Ouverte,tab2,how='inner',left_on=["jour_semaine","hour_trx","dataKey","stoAnabelKey"],right_on=[0,"Heure de début","2_x","anabel"])
tab3.sort_values(by=['mag_cod_magasin', 'jour_semaine','DEBUT'])
df = tab3[['mag_cod_magasin','jour_semaine','DEBUT',4,6,6,6,6,6,'Nb_caisses_ouvertes',6,"taux sco"]]
df = df.set_axis(['Anabel', 'Jour','Heure','Caisses_Prevues','Caisses_Planifiees','Ecart_en_Nb_de_Caisses','Prevues_en_H','Planifiees_en_H','Ecart_en_H','Nb_de_Caisses_Ouvertes','Caisses_Ouvertes_en_H','taux sco'], axis=1)
#--------------------------------------------------------------------------------------------
for i in range(0,len(df[['Anabel']])):
  if isinstance(df.at[i, 'Caisses_Prevues'], str):
    df.at[i, 'Caisses_Prevues'] = df.at[i, 'Caisses_Prevues'].replace(",",".")
sous_couverture = []
sur_couverture = []
sur_sous_couverture = []
taux_sous_couverture = []
taux_sur_couverture = []

for i in range(0,len(df[['Anabel']])):

  df = df.astype({"Caisses_Prevues": float,"Caisses_Planifiees":float,"Ecart_en_Nb_de_Caisses": float,"Prevues_en_H":float,"Planifiees_en_H": float,"Ecart_en_H":float,"Nb_de_Caisses_Ouvertes": float,"Caisses_Ouvertes_en_H":float})
  #-----------------------------------------------------------------------------------------------
  df.at[i, 'Ecart_en_Nb_de_Caisses'] = df.at[i, 'Caisses_Planifiees']- df.at[i, 'Caisses_Prevues']
  df.at[i, 'Prevues_en_H'] = df.at[i, 'Caisses_Prevues']*0.25
  df.at[i, 'Planifiees_en_H'] = df.at[i, 'Caisses_Planifiees']*0.25
  df.at[i, 'Ecart_en_H'] = df.at[i, 'Planifiees_en_H']- df.at[i, 'Prevues_en_H']
  df.at[i, 'Caisses_Ouvertes_en_H'] = df.at[i, 'Nb_de_Caisses_Ouvertes']*0.25
  #-----------------------------------------------------------------------------------------------
  if (df.at[i, 'Prevues_en_H'] - df.at[i, 'Caisses_Ouvertes_en_H'] > 0):
    sur_couverture.append(0)
    taux_sur_couverture.append(0)
    sous_couverture.append(df.at[i, 'Prevues_en_H'] - df.at[i, 'Caisses_Ouvertes_en_H'])
    sur_sous_couverture.append(-(df.at[i, 'Prevues_en_H'] - df.at[i, 'Caisses_Ouvertes_en_H']))
    taux_sous_couverture.append(sous_couverture[i] / df.at[i, 'Prevues_en_H'] )
  else:
    if  (df.at[i, 'Prevues_en_H'] - df.at[i, 'Caisses_Ouvertes_en_H'] < 0):
      sur_couverture.append(df.at[i, 'Caisses_Ouvertes_en_H'] - df.at[i, 'Prevues_en_H'])
      sur_sous_couverture.append( df.at[i, 'Caisses_Ouvertes_en_H'] - df.at[i, 'Prevues_en_H'])
      taux_sur_couverture.append(sur_couverture[i] / df.at[i, 'Caisses_Ouvertes_en_H'] )
      sous_couverture.append(0)
      taux_sous_couverture.append(0)

    else:
      sur_sous_couverture.append(0)
      sous_couverture.append(0)
      sur_couverture.append(0)
      taux_sur_couverture.append(0)
      taux_sous_couverture.append(0)


df['Ecart_sur_sous_couverture']= sur_sous_couverture
df['Ecart_sous_couverture']= sous_couverture
df['Ecart_sur_couverture']= sur_couverture
df['Taux_sur_couverture']= taux_sur_couverture
df['Taux_sous_couverture']= taux_sous_couverture

#-------------------------------------------------------------------------------------------------
link7 = "/content/Attente.csv"
#link7 = "/content/drive/MyDrive/Direction Encaissement/{}/{}/Attente.csv".format(semaine,jour)

#Attente_en_caisse        = pd.read_csv(link6, sep=",",encoding='latin-1')
#Attente_en_caisse_eclate = pd.read_csv(link7, sep=",",encoding="utf-8")



#---------------------------------------------------------------------------------------
#-----------------    on a Remplacer les données brutes par notre picasso      ---------
import datetime
linkpicasso = "/content/Attente.csv"
picasso               = pd.read_csv(link7, sep=",", header = 0)


column_names = ['creationDate','Magasin (code Anabel)','Magasin','QuarterId','pointOfSaleSurveyResponseValueInt']
column_names_2 = []
for col in picasso.columns:
  column_names_2.append(col)

newpicasso = pd.DataFrame(columns=column_names)

#Convert date format from picasso to much Darwin's date
#date = datetime.datetime.strptime("24/11/2021",'%d/%m/%Y')	
#date = datetime.datetime.strptime(str(date),"%Y-%m-%d %H:%M:%S").strftime('%d %b. %Y')


#Creating the new picasso DB
for k in range(0,len(picasso[['Magasin']])):
  magasin = picasso.at[k, "Magasin"]
  date = datetime.datetime.strptime(str(picasso.at[k, "Date"]),'%d/%m/%Y')	
  date = datetime.datetime.strptime(str(date),"%Y-%m-%d %H:%M:%S").strftime('%d %b. %Y')
  anabel = picasso.at[k, "Anabel"]
  valeurs = list(picasso.iloc[k,4:56].values)

  #Convert time format from picasso to much Darwin's time
  #Generating the time as a row instead of a column 
  for i in range(4,56):
    l = len(newpicasso)
    newpicasso.at[l, "QuarterId"] = column_names_2[i][0:5]
    newpicasso.at[l, "creationDate"] = date
    newpicasso.at[l, "Magasin (code Anabel)"] = anabel
    newpicasso.at[l, "Magasin"] = magasin
    newpicasso.at[l, "pointOfSaleSurveyResponseValueInt"] = valeurs[i-4]

Attente_en_caisse_eclate = newpicasso.copy()
#-----------------------------------------------------------------------------------------
for i in range(0,len(Attente_en_caisse_eclate[['QuarterId']])):
  if len(Attente_en_caisse_eclate.at[i, "QuarterId"])== 5:
    Attente_en_caisse_eclate.at[i, "QuarterId"] = Attente_en_caisse_eclate.at[i, "QuarterId"]+":00"
BDD = pd.merge(df,Attente_en_caisse_eclate,how='inner',left_on=['Anabel','Heure'],right_on=['Magasin (code Anabel)','QuarterId'])
BDD = BDD[['Anabel','Magasin','Jour','creationDate','Heure','Caisses_Prevues','Caisses_Planifiees','Ecart_en_Nb_de_Caisses','Prevues_en_H','Planifiees_en_H','Ecart_en_H','Nb_de_Caisses_Ouvertes','Caisses_Ouvertes_en_H','Ecart_sous_couverture','Taux_sous_couverture','Ecart_sur_couverture','Taux_sur_couverture','Ecart_sur_sous_couverture','pointOfSaleSurveyResponseValueInt','taux sco']]
BDD = BDD.rename(columns={"pointOfSaleSurveyResponseValueInt":"attente_en_caisse"})
BDD = BDD.rename(columns={"taux sco":"Taux SCO"})

BDD["3plus"] = BDD["attente_en_caisse"]

for i in range(0,len(BDD[['3plus']])):
  if BDD.at[i, "3plus"]<3 :
    BDD.at[i, "3plus"] = 0
#-----------------------------------------------------------------------------------------

#link8 = "/content/Anomalies.csv"
#link8 = "/content/drive/MyDrive/Direction Encaissement/{}/{}/Anomalies.csv".format(semaine,jour)

#Anomalies               = pd.read_csv(link8, sep=",", header = 0)
#Anomalies = Anomalies[['Magasin','% Anomalies','% Illisibles','% Inconnus']]
#for i in range(0,len(Anomalies[['Magasin']])):
#    Anomalies.at[i, 'Anabel'] = Anomalies.at[i,'Magasin'][:4]
#Anomalies = Anomalies[['Anabel','% Anomalies','% Illisibles','% Inconnus']]

#Anomalies = Anomalies.astype({'Anabel': int})
#BDD1 = pd.merge(BDD,Anomalies,how='inner',left_on=['Anabel'],right_on=['Anabel'])

#-------------------------------------------------------------------------------------------------
linkbdd = 'BDD_{}_{}.csv'.format(semaine,jour)
BDD.to_csv(linkbdd, index = False, header=True,sep=';',encoding="latin-1")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


KeyError: ignored